In [24]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import re

import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud
import pyLDAvis
sns.set()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim


# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords

In [25]:
df = pd.read_csv('df_description_processed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379 entries, 0 to 5378
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Description         5379 non-null   object
 1   lower_description   5379 non-null   object
 2   word_tokenized      5379 non-null   object
 3   sentence_tokenized  5379 non-null   object
 4   word_count          5379 non-null   int64 
 5   sentence_count      5379 non-null   int64 
 6   clean_words         5379 non-null   object
 7   clean_stemmed       5379 non-null   object
 8   clean_lemmed        5379 non-null   object
dtypes: int64(2), object(7)
memory usage: 378.3+ KB


In [48]:
# create the transform
vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.75)
# tokenize and build vocab
bag_of_words = vectorizer.fit_transform(df.clean_lemmed)
# summarize
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:10]

[('business', 14210),
 ('analysis', 9052),
 ('skill', 8872),
 ('analytics', 8459),
 ('ability', 8236),
 ('science', 7638),
 ('solution', 7225),
 ('learning', 7205),
 ('technology', 6725),
 ('development', 6163)]

In [49]:
# 5379 documents represented as a 14,409 dimensional vector (14,409 words)
bag_of_words

<5379x18641 sparse matrix of type '<class 'numpy.int64'>'
	with 899940 stored elements in Compressed Sparse Row format>

In [50]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(bag_of_words)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [51]:
first_topic = LDA.components_[0]

In [52]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([15594, 16104,  1515, 13446, 11054,  1503,  2910, 10359, 15081,
        9969], dtype=int64)

In [53]:
for i in top_topic_words:
    print(vectorizer.get_feature_names()[i])

skill
statistical
analytics
product
model
analysis
business
machine
science
learning


In [54]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['skill', 'statistical', 'analytics', 'product', 'model', 'analysis', 'business', 'machine', 'science', 'learning']


Top 10 words for topic #1:
['process', 'report', 'analytics', 'support', 'project', 'skill', 'management', 'ability', 'analysis', 'business']


Top 10 words for topic #2:
['state', 'science', 'veteran', 'disability', 'business', 'analytics', 'employment', 'technology', 'status', 'solution']


Top 10 words for topic #3:
['technology', 'new', 'help', 'customer', 'product', 'benefit', 'employee', 'business', 'opportunity', 'company']


Top 10 words for topic #4:
['working', 'solution', 'knowledge', 'software', 'tool', 'sql', 'database', 'development', 'technology', 'design']




In [55]:
topic_values = LDA.transform(bag_of_words)
topic_values.shape

(5379, 5)

In [56]:
df['Topic'] = topic_values.argmax(axis=1)

In [57]:
df.head()

,Description,lower_description,word_tokenized,sentence_tokenized,word_count,sentence_count,clean_words,clean_stemmed,clean_lemmed,Topic
0,"POSITION SUMMARY, The Business Analyst role is...","position summary, the business analyst role is...","['position', 'summary', 'the', 'business', 'an...","['POSITION SUMMARY, The Business Analyst role ...",424,25,"['position', 'summary', 'business', 'analyst',...","['posit', 'summari', 'busi', 'analyst', 'role'...","['position', 'summary', 'business', 'analyst',...",3
1,"What do we need?, You to have an amazing perso...","what do we need?, you to have an amazing perso...","['what', 'do', 'we', 'need', 'you', 'to', 'hav...","['What do we need?, You to have an amazing per...",286,10,"['need', 'amazing', 'personality', 'communicat...","['need', 'amaz', 'person', 'commun', 'style', ...","['need', 'amazing', 'personality', 'communicat...",1
2,"Validate, analyze, and conduct statistical ana...","validate, analyze, and conduct statistical ana...","['validate', 'analyze', 'and', 'conduct', 'sta...","['Validate, analyze, and conduct statistical a...",314,24,"['validate', 'analyze', 'conduct', 'statistica...","['valid', 'analyz', 'conduct', 'statist', 'ana...","['validate', 'analyze', 'conduct', 'statistica...",1
3,"Full time, Washington, DC metro area, Starting...","full time, washington, dc metro area, starting...","['full', 'time', 'washington', 'dc', 'metro', ...","['Full time, Washington, DC metro area, Starti...",297,13,"['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ...",0
4,Assist in consultations with business partners...,assist in consultations with business partners...,"['assist', 'in', 'consultations', 'with', 'bus...",['Assist in consultations with business partne...,316,7,"['assist', 'consultations', 'business', 'partn...","['assist', 'consult', 'busi', 'partner', 'inte...","['assist', 'consultation', 'business', 'partne...",1
